## Temat 2
Przewidywanie wzięcia pożyczki. 
Celem projektu jest zastosowanie modeli klasyfikacji (binarnej) do
przewidzenia czy pożyczka zostanie udzielona danemu klientowi z danymi parametrami wniosku.

Link do danych:
https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv
 <br>Opis danych (najdokładniejszy jaki istnieje):
https://discuss.analyticsvidhya.com/t/hackathon-3-x-predict-customer-worth-for-happy-customer-bank/3802
 <br>Zmienna celu (wypłacenie pożyczki): Disbursed

Wymogi:
  * Zmienna LoggedIn nie może być uwzględniona w modelowaniu - należy ją od razu wyrzucić ze zbioru.
  * Nie usuwamy żadnych obserwacji - braki danych uzupełniamy w jakiś sposób.
  * Wszelkie nieoczywiste ​ przekształcenia danych należy opatrzyć uzasadnieniem​ dlaczego dokonujemy tego przekształcenia (np. robimy wykres i uzasadniamy wzięci logarytmu ze zmiennej tym, że rozkład jest skośny).
  * Należy wykorzystać przynajmniej 3 metody klasyfikacji.
  * Należy uwzględnić optymalizację modeli/pipelinów.
  * Projekt musi być zakończony przejrzystym porównaniem przetestowanych rozwiązań(modeli/pipelinów) ​ w postaci tabeli​ , gdzie jeden wiersz opisuje jedno rozwiązanie i jego

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import BaggingClassifier
from xgboost.sklearn import XGBClassifier

In [2]:
# ściąram dane
data = pd.read_csv("https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv", 
                   encoding = "ISO-8859-1")
#na_values="?"

data.head()

,ID,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,LoggedIn,Disbursed
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,...,NaN,NaN,NaN,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,...,NaN,NaN,NaN,N,Web-browser,B,S134,3,1,0


In [3]:
# sprawdzam jak wyglądają dane
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87020 entries, 0 to 87019
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     87020 non-null  object 
 1   Gender                 87020 non-null  object 
 2   City                   86017 non-null  object 
 3   Monthly_Income         87020 non-null  int64  
 4   DOB                    87020 non-null  object 
 5   Lead_Creation_Date     87020 non-null  object 
 6   Loan_Amount_Applied    86949 non-null  float64
 7   Loan_Tenure_Applied    86949 non-null  float64
 8   Existing_EMI           86949 non-null  float64
 9   Employer_Name          86949 non-null  object 
 10  Salary_Account         75256 non-null  object 
 11  Mobile_Verified        87020 non-null  object 
 12  Var5                   87020 non-null  int64  
 13  Var1                   87020 non-null  object 
 14  Loan_Amount_Submitted  52407 non-null  float64
 15  Lo

In [4]:
# upewniam się co do zawartości zmiennej celu
data.Disbursed.value_counts()

0    85747
1     1273
Name: Disbursed, dtype: int64

In [5]:
# usuwam kolumnę ID i LoggedIn - zgodnie z zaleceniami
data.drop(["ID","LoggedIn"], axis=1, inplace=True)

In [6]:
# sprawdzam co jest w kolumnie Gender
data.Gender.value_counts()

Male      49848
Female    37172
Name: Gender, dtype: int64

In [7]:
# zamieniam na na inty (0 i 1)
data["Gender"] = (data.Gender == "Female").astype(int)

In [8]:
data.Gender.value_counts()

0    49848
1    37172
Name: Gender, dtype: int64

In [9]:
# sprawdzam co jest w kolumnie City
data.City.value_counts()

Delhi        12527
Bengaluru    10824
Mumbai       10795
Hyderabad     7272
Chennai       6916
             ...  
Chandel          1
IDAR             1
KAPADWANJ        1
RADHANPUR        1
DHANDHUKA        1
Name: City, Length: 697, dtype: int64

In [10]:
# uzupełniam braki dodatkową zmienną
data["City"] = data.City.fillna("X", inplace=False)

In [11]:
# sprawdzam co jest w kolumnie Monthly_Income
data.Monthly_Income

0         20000
1         35000
2         22500
3         35000
4        100000
          ...  
87015     71901
87016     16000
87017    118000
87018     98930
87019     42300
Name: Monthly_Income, Length: 87020, dtype: int64

In [12]:
# podglądam rozkład danych
data.Monthly_Income.describe()

count    8.702000e+04
mean     5.884997e+04
std      2.177511e+06
min      0.000000e+00
25%      1.650000e+04
50%      2.500000e+04
75%      4.000000e+04
max      4.445544e+08
Name: Monthly_Income, dtype: float64

In [13]:
# robię wiek z daty
def calculate_age(born):
    born = datetime.strptime(born, '%d-%b-%y').date()
    today = date(2015, 1, 1)
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
data['age'] = data['DOB'].apply(calculate_age)

In [14]:
# sprawdzam czy wiek z daty jest ok
data.head()

,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,Salary_Account,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,Disbursed,age
0,1,Delhi,20000,23-May-78,15-May-15,300000.0,5.0,0.0,CYBOSOL,HDFC Bank,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,36
1,0,Mumbai,35000,07-Oct-85,04-May-15,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,29
2,0,Panchkula,22500,10-Oct-81,19-May-15,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,33
3,0,Saharsa,35000,30-Nov-87,09-May-15,1000000.0,5.0,0.0,BIHAR GOVERNMENT,State Bank of India,...,NaN,NaN,NaN,N,Web-browser,B,S143,3,0,27
4,0,Bengaluru,100000,17-Feb-84,20-May-15,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,...,NaN,NaN,NaN,N,Web-browser,B,S134,3,0,30


In [15]:
# sprawdzam czy wiek z daty jest ok
data.age

0        36
1        29
2        33
3        27
4        30
         ..
87015    45
87016    24
87017    42
87018    37
87019    26
Name: age, Length: 87020, dtype: int64

In [16]:
# usuwam datę urodzenia
data.drop(["DOB"], axis=1, inplace=True)

In [17]:
# wyciągam tylko dzień złożenia wniosku, może to ma jakiś wpływ
data["Day_loan"] = pd.to_datetime(data.Lead_Creation_Date).apply(lambda x: x.day)

In [18]:
# sprawdzam jak rozkładają się dane na podstawie dnia złożenia wniosku
data.Day_loan.value_counts()

3     3908
23    3867
27    3400
18    3315
22    3310
29    3199
15    3183
4     3095
20    3049
30    3039
26    2910
6     2859
8     2851
16    2847
21    2812
28    2770
9     2743
17    2716
13    2685
2     2659
25    2655
5     2562
7     2507
1     2469
12    2465
11    2383
14    2295
24    2282
19    2257
10    2162
31    1766
Name: Day_loan, dtype: int64

In [19]:
# usuwam Lead_Creation_Date
data.drop(["Lead_Creation_Date"], axis=1, inplace=True)

In [20]:
#sprawdzam co z danymi
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87020 entries, 0 to 87019
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Gender                 87020 non-null  int32  
 1   City                   87020 non-null  object 
 2   Monthly_Income         87020 non-null  int64  
 3   Loan_Amount_Applied    86949 non-null  float64
 4   Loan_Tenure_Applied    86949 non-null  float64
 5   Existing_EMI           86949 non-null  float64
 6   Employer_Name          86949 non-null  object 
 7   Salary_Account         75256 non-null  object 
 8   Mobile_Verified        87020 non-null  object 
 9   Var5                   87020 non-null  int64  
 10  Var1                   87020 non-null  object 
 11  Loan_Amount_Submitted  52407 non-null  float64
 12  Loan_Tenure_Submitted  52407 non-null  float64
 13  Interest_Rate          27726 non-null  float64
 14  Processing_Fee         27420 non-null  float64
 15  EM

In [21]:
# # podglądam co tam w danych
data.Loan_Amount_Applied

0         300000.0
1         200000.0
2         600000.0
3        1000000.0
4         500000.0
           ...    
87015    1000000.0
87016          0.0
87017          0.0
87018     800000.0
87019          0.0
Name: Loan_Amount_Applied, Length: 87020, dtype: float64

In [22]:
# mały brak - 70, podobny do sąsienich kategorii, uzupełniam braki dodatkową zmienną, bez naczenia dla całości
data["Loan_Amount_Applied"] = data.Loan_Amount_Applied.fillna(0)
data["Loan_Tenure_Applied"] = data.Loan_Tenure_Applied.fillna(0)
data["Existing_EMI"] = data.Existing_EMI.fillna(0)

In [23]:
# uzupełniam braki dodatkową zmienną
data["Employer_Name"] = data.Employer_Name.fillna("X")

In [24]:
# uzupełniam braki dodatkową zmienną
data.Salary_Account.fillna("X", inplace=True)

In [25]:
# podglądam co tam w danych
data.Salary_Account.value_counts()

HDFC Bank                                          17695
ICICI Bank                                         13636
State Bank of India                                11843
X                                                  11764
Axis Bank                                           8783
Citibank                                            2376
Kotak Bank                                          2067
IDBI Bank                                           1550
Punjab National Bank                                1201
Bank of India                                       1170
Bank of Baroda                                      1126
Standard Chartered Bank                              995
Canara Bank                                          990
Union Bank of India                                  951
Yes Bank                                             779
ING Vysya                                            678
Corporation bank                                     649
Indian Overseas Bank           

In [26]:
# sprawdzam co tam siedzi
data.Mobile_Verified.value_counts()

Y    56481
N    30539
Name: Mobile_Verified, dtype: int64

In [27]:
# zamieniam na na inty (0 i 1)
data["Mobile_Verified"] = (data.Mobile_Verified == "Y").astype(int)

In [28]:
data.Mobile_Verified.value_counts()

1    56481
0    30539
Name: Mobile_Verified, dtype: int64

In [29]:
# sprawdzam co tam siedzi
data.Var5.value_counts()

0     29087
1     12236
3      6759
11     5204
2      4485
14     3662
15     3509
12     2989
13     2622
8      2515
10     2427
9      2281
16     2097
4      1815
17     1691
7      1489
6       983
5       975
18      194
Name: Var5, dtype: int64

In [30]:
# sprawdzam co tam siedzi
data.Var1.value_counts()

HBXX    59294
HBXC     9010
HBXB     4479
HAXA     2909
HBXA     2123
HAXB     2011
HBXD     1964
HAXC     1536
HBXH      970
HCXF      722
HAYT      508
HAVC      384
HAXM      268
HCXD      237
HCYS      217
HVYS      186
HAZD      109
HCXG       78
HAXF       15
Name: Var1, dtype: int64

In [31]:
# sprawdzam co tam siedzi
data.Loan_Amount_Submitted.value_counts()

100000.0     6884
200000.0     6583
300000.0     5385
500000.0     4849
1000000.0    1644
             ... 
2470000.0       1
1660000.0       1
1890000.0       1
1990000.0       1
1900000.0       1
Name: Loan_Amount_Submitted, Length: 203, dtype: int64

In [32]:
# sprawdzam czy nie ma ujemnych wartości
np.sum(data.Loan_Amount_Submitted<0)

0

In [33]:
# wstawiam -1 jako nową zmienną
data.Loan_Amount_Submitted.fillna(-1, inplace=True)

In [34]:
data.Loan_Amount_Submitted

0             -1.0
1         200000.0
2         450000.0
3         920000.0
4         500000.0
           ...    
87015         -1.0
87016     240000.0
87017    1200000.0
87018     800000.0
87019     690000.0
Name: Loan_Amount_Submitted, Length: 87020, dtype: float64

In [35]:
# sprawdzam co tam siedzi
data.Loan_Tenure_Submitted.value_counts()

5.0    20765
4.0    15135
3.0     8858
2.0     5332
1.0     2314
6.0        3
Name: Loan_Tenure_Submitted, dtype: int64

In [36]:
# sprawdzam czy nie ma ujemnych wartości
np.sum(data.Loan_Tenure_Submitted<0)

0

In [37]:
# wstawiam -1 jako nową zmienną
data.Loan_Tenure_Submitted.fillna(-1, inplace=True)

In [38]:
data.Loan_Tenure_Submitted.value_counts()

-1.0    34613
 5.0    20765
 4.0    15135
 3.0     8858
 2.0     5332
 1.0     2314
 6.0        3
Name: Loan_Tenure_Submitted, dtype: int64

In [39]:
# Interest_Rate, Processing_Fee, EMI_Loan_Submitted, wstawiam -1 jako nową zmienną
data.Interest_Rate.fillna(-1, inplace=True)
data.Processing_Fee.fillna(-1, inplace=True)
data.EMI_Loan_Submitted.fillna(-1, inplace=True)

In [40]:
data.Filled_Form.value_counts()

N    67530
Y    19490
Name: Filled_Form, dtype: int64

In [41]:
# zamieniam na na inty (0 i 1)
data["Filled_Form"] = (data.Filled_Form=="Y").astype(int)

In [42]:
data.Filled_Form.value_counts()

0    67530
1    19490
Name: Filled_Form, dtype: int64

In [43]:
data.Device_Type.value_counts()

Web-browser    64316
Mobile         22704
Name: Device_Type, dtype: int64

In [44]:
# zamieniam na na inty (0 i 1)
data["Device_Type"] = (data.Device_Type=="Mobile").astype(int)

In [45]:
data.Device_Type.value_counts()

0    64316
1    22704
Name: Device_Type, dtype: int64

In [46]:
# one-hot encoding
# X = pd.get_dummies(data.drop(["Disbursed"], axis=1, inplace=False))
# y = data.Disbursed

In [47]:
# X.shape (87020, 44396)

In [48]:
# kolumn ogromnie dużo i niewiele to wniesie. Wrzucę mniejsze ilości występowań do jednego worka
emp_list = data["Employer_Name"].value_counts()[:10].index.tolist()
data['Employer_Name'] = data['Employer_Name'].apply(lambda x: x if x in emp_list else 'Other')

In [49]:
city_list = data["City"].value_counts()[:10].index.tolist()
data['City'] = data['City'].apply(lambda x: x if x in city_list else 'Other')

In [50]:
# one-hot encoding
X = pd.get_dummies(data.drop(["Disbursed"], axis=1, inplace=False))
y = data.Disbursed

In [51]:
X.shape

(87020, 153)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

In [53]:
models = [
    Pipeline([("scaler", StandardScaler()),
              ("model", LogisticRegression())]),
    Pipeline([("model", DecisionTreeClassifier())]),
    GaussianNB(),
    Pipeline([("scaler", MaxAbsScaler()),
              ("model", SVC())]),
    Pipeline([("scaler", MaxAbsScaler()),
              ("model", LinearSVC())]),
    BaggingClassifier(DecisionTreeClassifier(),
                          n_estimators=100,
                          max_samples=0.5),
    XGBClassifier(max_depth=5)
]

params = [
    {"model__C":[0.01, 0.1, 1, 10, 100],
     "model__penalty":["l2", "l1"]},
    
    {"model__min_samples_leaf":[10, 25, 50, 100],
     "model__max_depth":[None, 10, 20]},
    
    {},
    
    #[{"model__kernel":["rbf"], 
      #"model__gamma":[0.1,1,10],
      #"model__C":[0.1, 1, 10]},
     {"model__kernel":["poly"], 
      "model__degree":[2,3],
      "model__C":[0.1, 1, 10]},
    
    {"model__C":[0.01, 0.1, 1, 10, 100]},
    
    {"base_estimator__min_samples_leaf":[10, 25, 50],
     "base_estimator__max_depth":[None, 10, 20]},
    {"n_estimators":[20, 40],
     "learning_rate":[0.01, 0.05, 0.1, 0.2, 0.3]}
]


for model, param_grid in zip(models, params):
    optimizer = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
    optimizer.fit(X_train, y_train)
    print(accuracy_score(y_test, 
                         optimizer.best_estimator_.predict(X_test)))
    


0.9866697310962997
0.9866697310962997
0.8324523098138359
0.9866697310962997
0.9866697310962997
0.9866697310962997
0.9866697310962997


In [57]:
print('Porównanie')

df = pd.DataFrame({
    'Model' : ['LogisticRegression', 'DecisionTreeClassifier', 'GaussianNB', 'SVC', 'LinearSVC', 'BaggingClassifier', 'XGBClassifier'],
    'Accuracy Score' : [0.98, 0.98, 0.82, 0.98, 0.98, 0.98,0.98]
})
df

Porównanie


,Model,Accuracy Score
0,LogisticRegression,0.98
1,DecisionTreeClassifier,0.98
2,GaussianNB,0.82
3,SVC,0.98
4,LinearSVC,0.98
5,BaggingClassifier,0.98
6,XGBClassifier,0.98
